In [41]:
#f = open('Day_11_example.txt')
f = open('Day_11_input.txt')
paragraphs = [p.splitlines() for p in f.read().split('\n\n')]
paragraphs

[['Monkey 0:',
  '  Starting items: 83, 97, 95, 67',
  '  Operation: new = old * 19',
  '  Test: divisible by 17',
  '    If true: throw to monkey 2',
  '    If false: throw to monkey 7'],
 ['Monkey 1:',
  '  Starting items: 71, 70, 79, 88, 56, 70',
  '  Operation: new = old + 2',
  '  Test: divisible by 19',
  '    If true: throw to monkey 7',
  '    If false: throw to monkey 0'],
 ['Monkey 2:',
  '  Starting items: 98, 51, 51, 63, 80, 85, 84, 95',
  '  Operation: new = old + 7',
  '  Test: divisible by 7',
  '    If true: throw to monkey 4',
  '    If false: throw to monkey 3'],
 ['Monkey 3:',
  '  Starting items: 77, 90, 82, 80, 79',
  '  Operation: new = old + 1',
  '  Test: divisible by 11',
  '    If true: throw to monkey 6',
  '    If false: throw to monkey 4'],
 ['Monkey 4:',
  '  Starting items: 68',
  '  Operation: new = old * 5',
  '  Test: divisible by 13',
  '    If true: throw to monkey 6',
  '    If false: throw to monkey 5'],
 ['Monkey 5:',
  '  Starting items: 60, 94',

In [55]:
from dataclasses import dataclass
from typing import Callable

@dataclass
class Monkey:
    items: list[int]
    operation: Callable[[int], int]
    divisor: int
    if_true: int
    if_false: int
    inspected_items: int
    
    @classmethod
    def parse(cls, lines):
        assert lines[0].startswith('Monkey ')
        assert lines[1].startswith('  Starting items: ')
        items = [int(item.strip()) for item in lines[1][18:].split(',')]
        assert lines[2].startswith('  Operation: new = old ')
        op, arg = lines[2][23:].split(' ')
        if op == '*' and arg == 'old':
            operation = lambda old: old * old
        elif op == '*':
            operation = lambda old: old * int(arg)
        elif op == '+':
            operation = lambda old: old + int(arg)
        else:
            assert False
        assert lines[3].startswith('  Test: divisible by ')
        divisor = int(lines[3][21:])
        assert lines[4].startswith('    If true: throw to monkey ')
        if_true = int(lines[4][29:])
        assert lines[5].startswith('    If false: throw to monkey ')
        if_false = int(lines[5][30:])
        
        return Monkey(items, operation, divisor, if_true, if_false, 0)
    
    def round(self, worry_divisor):
        throws = []
        while self.items:
            item = self.items.pop(0) 
            new = self.operation(item) // worry_divisor
            if new % self.divisor == 0:
                throws.append((self.if_true, new))
            else:
                throws.append((self.if_false, new))
            self.inspected_items += 1
        return throws

    def round2(self, worry_divisor, mod_product):
        throws = []
        while self.items:
            item = self.items.pop(0) 
            new = (self.operation(item) // worry_divisor) % mod_product
            if new % self.divisor == 0:
                throws.append((self.if_true, new))
            else:
                throws.append((self.if_false, new))
            self.inspected_items += 1
        return throws
         

In [56]:
monkeys = [Monkey.parse(p) for p in paragraphs]

In [57]:
for _ in range(20):
    for monkey in monkeys:
        throws = monkey.round(3)
        for m, item in throws:
            monkeys[m].items.append(item)

In [58]:
from math import prod
def monkey_business(monkeys):
    return prod(sorted([m.inspected_items for m in monkeys])[-2:])

In [59]:
# Part 1 solution
monkey_business(monkeys)

108240

In [62]:
monkeys = [Monkey.parse(p) for p in paragraphs]

In [63]:
mod_product = prod([m.divisor for m in monkeys])
for _ in range(10000):
    for monkey in monkeys:
        throws = monkey.round2(1, mod_product)
        for m, item in throws:
            monkeys[m].items.append(item)

In [64]:
# Part 2 solution
monkey_business(monkeys)

25712998901